In [3]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")   # se notebook sta in notebooks/ e src/ è a fianco

import src.models as md
import src.features as ft

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
from src.data import load_train_set

X_train, X_val, y_train, y_val = load_train_set()

## 2. Feature Selection

In [5]:
importance_dfs = ft.get_feature_importances(md.MODELS, X_train, y_train)
importance_df = ft.combine_feature_importances(importance_dfs)

print(importance_df.shape)
print(len(importance_df))
importance_df.head()

Values to compute: .
(66, 7)
66


,feature,importance_xgb,importance_ctb,importance_lgb,importance_svr,importance_mlp,importance_knn
33,1stFlrSF,0.050795,0.055531,0.047587,0.077864,0.126215,0.032349
34,2ndFlrSF,0.014597,0.026175,0.018163,0.062962,0.112680,0.018871
59,3SsnPorch,0.000000,0.000007,0.000000,0.000000,0.000000,0.000000
41,BedroomAbvGr,0.002850,0.008655,0.001769,0.004588,0.013974,0.008968
7,BldgType,0.003606,0.003777,0.000374,0.000508,0.009924,0.004133


In [6]:
comparison_df = ft.get_feature_comparison(importance_df)

ft.plot_importance_comparison(comparison_df)
ft.plot_rank_comparison(comparison_df)

In [60]:
features_by_importance = list(comparison_df.sort_values('importance_sum', ascending=False)['feature'])

In [61]:
feature_numbers = [10, 15, 20, 25, 30, 35]
scores = ft.evaluate_feature_number_per_models(md.MODELS, feature_numbers, X_train, y_train, features_by_importance)

Computing cross validation scores for xgb.
Computing cross validation scores for ctb.
Learning rate set to 0.038982
0:	learn: 0.9543882	total: 1.15ms	remaining: 1.15s
1:	learn: 0.9287610	total: 1.63ms	remaining: 812ms
2:	learn: 0.9047567	total: 2.05ms	remaining: 682ms
3:	learn: 0.8829023	total: 2.53ms	remaining: 630ms
4:	learn: 0.8594271	total: 2.87ms	remaining: 571ms
5:	learn: 0.8391454	total: 3.33ms	remaining: 551ms
6:	learn: 0.8183111	total: 3.83ms	remaining: 543ms
7:	learn: 0.7999363	total: 4.24ms	remaining: 525ms
8:	learn: 0.7801219	total: 4.75ms	remaining: 523ms
9:	learn: 0.7623700	total: 5.09ms	remaining: 504ms
10:	learn: 0.7439513	total: 5.56ms	remaining: 500ms
11:	learn: 0.7274795	total: 6.07ms	remaining: 500ms
12:	learn: 0.7112659	total: 6.56ms	remaining: 498ms
13:	learn: 0.6956153	total: 7.04ms	remaining: 496ms
14:	learn: 0.6806909	total: 7.53ms	remaining: 494ms
15:	learn: 0.6660950	total: 7.87ms	remaining: 484ms
16:	learn: 0.6523709	total: 8.29ms	remaining: 479ms
17:	learn:

/opt/miniconda3/envs/torch_study/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:780: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.

/opt/miniconda3/envs/torch_study/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:780: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.

/opt/miniconda3/envs/torch_study/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:780: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.

/opt/miniconda3/envs/torch_study/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:780: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.

/opt/miniconda3/envs/torch_study/lib/python3.11/site-pac

Computing cross validation scores for knn.


In [62]:
import pandas as pd
import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objects as go

def plot_model_scores(scores, feature_numbers, items_per_row=2):
    # make the value_vars list once
    value_vars = [str(n) for n in feature_numbers] + ['all']

    n_models = len(scores)
    n_rows = (n_models + items_per_row - 1) // items_per_row

    # create subplot grid
    fig = sp.make_subplots(
        rows=n_rows,
        cols=items_per_row,
        subplot_titles=list(scores.keys())
    )

    for i, (model_name, df) in enumerate(scores.items()):
        # melt results for this model
        df_res = df.reset_index().melt(
            id_vars='index',
            value_vars=value_vars,
            var_name='number of features',
            value_name='score'
        )

        # compute mean/std across folds
        df_stats = df_res.groupby('number of features')['score'].agg(['mean', 'std']).reset_index()

        # add bar trace to correct subplot
        row = i // items_per_row + 1
        col = i % items_per_row + 1

        fig.add_trace(
            go.Bar(
                x=df_stats['number of features'],
                y=df_stats['mean'],
                error_y=dict(type='data', array=df_stats['std']),
                name=model_name
            ),
            row=row, col=col
        )

    # improve layout
    fig.update_layout(
        height=400 * n_rows,
        width=500 * items_per_row,
        showlegend=False
    )

    fig.show()
    
plot_model_scores(scores, feature_numbers, 3)
